In [2]:
import pandas as pd
import chardet
import random

# Load catalog info and scenarios file
with open('rag_corpus/ug_cat_metadata.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))  # Read first 10KB to detect
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")
catalog = pd.read_csv('./rag_corpus/ug_cat_metadata.csv', encoding=encoding)
catalog_majors = catalog[catalog['type'] == 'major']
scenarios =  pd.read_csv('evaluation/scenarios.csv')

# Iterate through each  row in the catalog mapping catalog years to valid degree-major-concentrations
degree_major_concentrations = {}
for catalog_year in catalog_majors['catalog_year'].unique():
    degree_major_concentrations[catalog_year] = []
    catalog_year_majors = catalog_majors[catalog_majors['catalog_year'] == catalog_year]

    for index, row in catalog_year_majors.iterrows():
        if row['file_name'][0:4] != 'BSET':
            degree_major_concentrations[catalog_year].append(row['file_name'])




catalog_year_credit_hours = {
    '2021-2022': ["60 to 89", "90 to 119", "120 to 149", "150 or more"],
    '2022-2023': ["30 to 59", "60 to 89", "90 to 119", "120 to 149", "150 or more"],
    '2023-2024': ["Up to 29", "30 to 59", "60 to 89", "90 to 119"],
    '2024-2025': ["Up to 29", "30 to 59", "60 to 89"],
    '2025-2026': ["None yet!", "Up to 29", "30 to 59"],
}

def assign_catalog_year(row):
    possible_years = []
    for year in catalog_year_credit_hours.keys():
        if row['credit_hours'] in catalog_year_credit_hours[year]:
            possible_years.append(year)  

    return random.choice(possible_years)

# Assign random degree major concentrations to each row in the scenarios DataFrame
def assign_degree_major_concentration(row):
    return random.choice(degree_major_concentrations[row['catalog_year']])



def assign_credit_hours(row):
    if row['set'] == 'Sophomore':
        return "30 to 59"
    elif row['set'] == 'Junior':
        return "60 to 89"
    elif row['set'] == 'Senior':
        return random.choice(["90 to 119", "120 to 149"]) 
    else:
        return random.choice(["None yet!", "Up to 29", "30 to 59", "60 to 89", "90 to 119", "120 to 149", "150 or more"])


scenarios['credit_hours'] = scenarios.apply(assign_credit_hours, axis=1)
scenarios['catalog_year'] = scenarios.apply(assign_catalog_year, axis=1)
scenarios['degree_major_concentration'] = scenarios.apply(assign_degree_major_concentration, axis=1)


scenarios.to_csv('evaluation/scenarios.csv', index=False)
scenarios.head()

Detected encoding: ISO-8859-1


,index,set,topic,user_prompt,credit_hours,catalog_year,degree_major_concentration
0,1,generic_1,Academic Planning & Course Selection,Which courses should I prioritize for next sem...,Up to 29,2023-2024,"BSCPE Computer Engineering, Machine Learning C..."
1,2,generic_1,Academic Planning & Course Selection,Are there any prerequisite chains I should be ...,Up to 29,2024-2025,BSME Mechanical Engineering
2,3,generic_1,Academic Planning & Course Selection,What's the recommended sequence for my remaini...,90 to 119,2021-2022,"BSEE Electrical Engineering, Power and Energy ..."
3,4,generic_1,Academic Planning & Course Selection,Should I take any courses over the summer to l...,Up to 29,2025-2026,BS Environmental Engineering
4,5,generic_1,Academic Planning & Course Selection,Which electives would best complement my caree...,Up to 29,2025-2026,BSME Mechanical Engineering
